In [2]:
import netCDF4 as nc
import os
import datetime

original_file_path = "/home/prateiksinha/new_data/mpi/tas/tas_6hrPlevPt_MPI-ESM1-2-HR_historical_r1i1p1f1_gn_199001010600-199501010000.nc"
original_dataset = nc.Dataset(original_file_path, "r")

FileNotFoundError: [Errno 2] No such file or directory: b'/home/prateiksinha/new_data/mpi/tas/tas_6hrPlevPt_MPI-ESM1-2-HR_historical_r1i1p1f1_gn_199001010600-199501010000.nc'

In [27]:
second_file_path = "/home/prateiksinha/new_data/mpi/tas/tas_6hrPlevPt_MPI-ESM1-2-HR_historical_r1i1p1f1_gn_199501010600-200001010000.nc"
second_dataset = nc.Dataset(second_file_path, "r")

In [8]:
import os
import netCDF4 as nc
import numpy as np
from datetime import datetime, timedelta


# Path to your original .nc file
input_nc_file = "/home/prateiksinha/new_data/mpi/tas/tas_6hrPlevPt_MPI-ESM1-2-HR_historical_r1i1p1f1_gn_199501010600-200001010000.nc"

# Directory to store the individual yearly .nc files
output_dir = "/home/prateiksinha/new_data/processed/mpi"

# Open the original .nc file for reading
with nc.Dataset(input_nc_file, "r") as ds:
    # Assuming the time variable is named "time" and has units in days since a reference date
    time_var = ds.variables["time"]
    
    # Convert time units to days since the reference date
    reference_date = datetime(1850, 1, 1, 0, 0, 0)
    time_days = time_var[:]
    
    print("TIME_DAYS", time_days)

    time_years = np.array([(reference_date + timedelta(days=float(days))).year for days in time_days])
    print("TIME_YEARS", time_years)
    
    # Loop through each year
    for year in range(int(time_years[0]), int(time_years[-1]) + 1):
        print("GOT THE YEAR", year)

        # Create a new .nc file for the current year
        output_nc_file = os.path.join(output_dir, f"climate_data_{year}.nc")
        with nc.Dataset(output_nc_file, "w") as output_ds:
            # Copy global attributes from the original file
            for attr_name in ds.ncattrs():
                setattr(output_ds, attr_name, getattr(ds, attr_name))
            
            # Create dimensions in the output file
            output_ds.createDimension("time", sum(year == time_years))  # Each year has one time step
            output_ds.createDimension("lat", len(ds.dimensions["lat"]))
            output_ds.createDimension("lon", len(ds.dimensions["lon"]))
            
            # Copy variables to the output file
            for var_name in ds.variables:
                if 'bnds' not in var_name:
                    var = ds.variables[var_name]
                    print("CURR VAR NAME", var_name)
                    print("ASSOCIATED VAR", dir(var))
                    output_var = output_ds.createVariable(var_name, var.dtype, var.dimensions)
                    
                    if "bnds" in var.dimensions:
                        # Handle bounds dimensions, e.g., for time variable
                        output_var[:] = var[:, year*2:year*2+2, :, :]
                    elif var_name == "time":
                        output_time_var = output_var
                        output_time_var.units = time_var.units
                        output_time_var.calendar = time_var.calendar
                        output_time_var[:] = time_days[time_years == year]  
                    elif var_name == 'tas':
                        output_var[:] = var[time_years == year]
                    else:
                        output_var[:] = var[:]
    
        print(f"Processed year {year}")

TIME_DAYS [52960.25 52960.5  52960.75 ... 54785.5  54785.75 54786.  ]
TIME_YEARS [1995 1995 1995 ... 1999 1999 2000]
GOT THE YEAR 1995
CURR VAR NAME time
ASSOCIATED VAR ['__array__', '__class__', '__delattr__', '__delitem__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__len__', '__lt__', '__ne__', '__new__', '__orthogonal_indexing__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setitem__', '__sizeof__', '__str__', '__subclasshook__', '_assign_vlen', '_check_safecast', '_cmptype', '_enumtype', '_get', '_getdims', '_getname', '_grp', '_grpid', '_has_lsd', '_iscompound', '_isenum', '_isprimitive', '_isvlen', '_name', '_ncstring_attrs__', '_nunlimdim', '_pack', '_put', '_toma', '_use_get_vars', '_varid', '_vltype', 'always_mask', 'assignValue', 'axis', 'calendar', 'chartostring', 'chunking', 'datatype', 'delncattr', 'dimensions', 'dtype', 'endian

In [7]:
original_file_path = "/home/prateiksinha/new_data/mpi/2m_temperature/1992.nc"
original_dataset = nc.Dataset(original_file_path, "r")
original_dataset.variables['tas']


<class 'netCDF4._netCDF4.Variable'>
float32 tas(time, lat, lon)
unlimited dimensions: 
current shape = (1464, 192, 384)
filling on, default _FillValue of 9.969209968386869e+36 used

In [6]:
original_file_path = "/home/prateiksinha/new_data/awi/2m_temperature/1992.nc"
original_dataset = nc.Dataset(original_file_path, "r")
original_dataset.variables['tas']

<class 'netCDF4._netCDF4.Variable'>
float32 tas(time, lat, lon)
    standard_name: air_temperature
    long_name: Near-Surface Air Temperature
    comment: near-surface (usually, 2 meter) air temperature
    units: K
    cell_methods: area: mean time: point
    cell_measures: area: areacella
    history: 2020-04-01T06:52:50Z altered by CMOR: Treated scalar dimension: 'height'. 2020-04-01T06:52:50Z altered by CMOR: replaced missing value flag (-9e+33) and corresponding data with standard missing value (1e+20). 2020-04-01T06:52:50Z altered by CMOR: Inverted axis: lat.
    coordinates: height
    missing_value: 1e+20
    _FillValue: 1e+20
unlimited dimensions: time
current shape = (1464, 96, 192)
filling on